In [ ]:
import matplotlib.pyplot as plt
import sympy as sp
import numpy as np
from IPython.display import display
from pyodesys.symbolic import SymbolicSys, PartiallySolvedSystem, symmetricsys
from pyodesys.tests.bateman import bateman_full
sp.init_printing()
%matplotlib inline

In [ ]:
logexp = (sp.log, sp.exp)
LogLogSys = symmetricsys(logexp, logexp)
LogDepSys = symmetricsys(logexp)

def integrate_and_plot(mysys, zero_time=0):
    y0 = [3, 2, 1]
    k = [3, 2, 1]
    atol = 1e-8
    oderes = mysys.integrate([zero_time, 1], y0, k, integrator='cvode', atol=atol, rtol=1e-10)
    plt.figure(figsize=(14,4))
    plt.subplot(1, 2, 1)
    x, y = mysys.plot_result()
    ref = np.asarray(bateman_full(y0, k, x, exp=np.exp)).T
    plt.subplot(1, 2, 2)
    aerr_over_atol = (y-ref)/atol
    for column, color, linestyle in zip(aerr_over_atol.T, 'krg', ('-', '--', ':')):
        plt.plot(x, column, c=color, ls=linestyle)
    plt.ylabel('Absolute error / Absolute tolerance')
    print({k: v for k, v in oderes[2].items() if k in 'nfev njev time_cpu'.split()})

def powsimp_proc(exprs):
    return [sp.powsimp(expr.expand(), force=True) for expr in exprs]

In [ ]:
odesys = SymbolicSys.from_callback(
    lambda x, y, p: [-p[0]*y[0], p[0]*y[0]-p[1]*y[1], p[1]*y[1]-p[2]*y[2]], 3, 3)
print("Linear system (ny=%d)" % odesys.ny)
display(odesys.exprs)
integrate_and_plot(odesys)

In [ ]:
partsys = PartiallySolvedSystem(odesys, lambda x0, y0, p0, be: {
        odesys.dep[0]: y0[0]*be.exp(-p0[0]*odesys.indep)})
print("Partially solved linear system (ny=%d)" % partsys.ny)
display(partsys.exprs)
integrate_and_plot(partsys)

In [ ]:
llsys = LogLogSys.from_other(odesys, exprs_process_cb=powsimp_proc)
print("Log/Log transformed system (ny=%d)" % llsys.ny)
display(llsys.exprs)
integrate_and_plot(llsys, zero_time=1e-10)

In [ ]:
llpartsys = LogLogSys.from_other(partsys, exprs_process_cb=powsimp_proc)
print("Log/Log transformed system (ny=%d)" % llpartsys.ny)
display(llpartsys.exprs)
integrate_and_plot(llpartsys, zero_time=1e-10)

In [ ]:
ldepsys = LogDepSys.from_other(odesys, exprs_process_cb=powsimp_proc)
print("ln(y) transformed system (ny=%d)" % ldepsys.ny)
display(ldepsys.exprs)
integrate_and_plot(ldepsys)

In [ ]:
ldeppartsys = LogDepSys.from_other(partsys, exprs_process_cb=powsimp_proc)
print("ln(y) transformed partially solved system (ny=%d)" % ldeppartsys.ny)
display(ldeppartsys.exprs)
integrate_and_plot(ldeppartsys)